In [1]:
import json
import numpy as np
from transforms3d.euler import euler2mat,mat2euler
import cv2

In [2]:
def get_extrinsic_matrix(pos, focus, up):
    forward = focus -  pos;forward = forward/np.linalg.norm(forward)
    left = np.cross(up,forward);left = left/np.linalg.norm(left)
    up  = np.cross(left,forward);up = up/np.linalg.norm(up)
    #print(forward)
    #print(left)
    #print(up)
    
    mat = np.zeros((3,3), dtype = np.float32)
    mat[0] = -left
    mat[1] = -up
    mat[2] = -forward
    return mat

In [3]:
def get_rvec_tvec(pos, focus, up):
    mat = get_extrinsic_matrix(pos, focus, up)
    rvec, _ = cv2.Rodrigues(mat)
    tvec = pos
    return rvec, tvec

In [4]:
def world2camera(pts_3d_world_homo, rvec, tvec):
    rmat, _ = cv2.Rodrigues(rvec)
    print(rmat)
    extrinsics = np.zeros((3,4))
    extrinsics[:,:3] = rmat
    extrinsics[:,3] = tvec.reshape(-1)
    
    pts_3d_cam = np.matmul(extrinsics, pts_3d_world_homo)
    
    return pts_3d_cam

In [5]:
def parse_xyz(pos):
    data = np.zeros((3),dtype = np.float64)
    data[0] = pos['x']
    data[1] = pos['y']
    data[2] = pos['z']
    return data
def add_w(pos):
    data = np.zeros((4),dtype = np.float64)
    data[0:3] = pos
    data[3] = 1
    return data

In [6]:
json_path = "/media/yalu/6066C1DD66C1B3D6/Github/DeepMimic/Output/data/9.json"

In [7]:
with open(json_path,"r") as file:
    data = json.load(file)

In [8]:
pos = parse_xyz(data["cam_par"]["pos"])
focus = parse_xyz(data["cam_par"]["focus"])
up = [0,1,0]

In [9]:
part_name = "right_ankle"
root_world = parse_xyz(data["WorldPos"][part_name])
root_camera = parse_xyz(data["CameraPos"][part_name])
print(root_camera)

[-0.11684408 -0.83263253 -4.93251413]


In [10]:
R0 = get_extrinsic_matrix(pos, focus, up)
print(R0)

[[ 6.123574e-17 -0.000000e+00 -1.000000e+00]
 [ 0.000000e+00  1.000000e+00 -0.000000e+00]
 [ 1.000000e+00 -0.000000e+00  6.123574e-17]]


In [11]:
answer0 = np.dot(R0,root_world) - np.dot(R0,pos)
np.allclose(answer0,root_camera)

True

In [12]:
rvec,tvec = get_rvec_tvec(pos, focus, up)
answer1 = world2camera(add_w(root_world), rvec, tvec)
print(answer1)

[[-4.371139e-08 -0.000000e+00 -1.000000e+00]
 [ 0.000000e+00  1.000000e+00 -0.000000e+00]
 [ 1.000000e+00  0.000000e+00 -4.371139e-08]]
[4.82327725 0.94083347 0.011112  ]


In [27]:
def Rmat(tx, ty, tz):
    tx = np.deg2rad(tx); ty = np.deg2rad(ty); tz = np.deg2rad(tz)
    
    Rx = np.zeros((3,3)); Ry = np.zeros((3,3)); Rz = np.zeros((3,3))
    
    Rx[0][0] = 1; Rx[1][1] = np.cos(tx); Rx[1][2] = -np.sin(tx); Rx[2][1] = np.sin(tx); Rx[2][2] = np.cos(tx)
    Ry[0][0] = np.cos(ty); Ry[0][2] = np.sin(ty); Ry[1][1] = 1; Ry[2][0] = -np.sin(ty); Ry[2][2] = np.cos(ty)    
    Rz[0][0] = np.cos(tz); Rz[1][1] = -np.sin(tz); Rz[1][0] = np.sin(tz); Rz[1][1] = np.cos(tx); Rz[2][2] = 1
    
    R = np.matmul(Rz, np.matmul(Ry, Rx))
    
    rvec,_ = cv2.Rodrigues(R)
    
    return rvec, R

In [28]:
rvec, mat = Rmat(0,90,0)
print(rvec)
print(mat)

[[0.        ]
 [1.57079633]
 [0.        ]]
[[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]


In [31]:
angle = [0,90,0]
angle = np.deg2rad(angle)
mat = euler2mat(angle[0],angle[1],angle[2])
print(mat)

[[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]


In [32]:
pos = [10,0,0]
angle = [0,180,0]
angle = np.deg2rad(angle)
mat = euler2mat(angle[0],angle[1],angle[2])
print(np.dot(mat,pos))

[-1.0000000e+01  0.0000000e+00 -1.2246468e-15]


In [35]:
def get_extrinsics(rvec,tvec):
    extMat = np.zeros((4,4))
    R = euler2mat(rvec[0],rvec[1],rvec[2],'rxyz')
    R = -np.flip(np.flip(R, axis=0), axis =1) * np.array([[1,-1,1],[-1,1,-1],[-1,1,-1]])
    extMat[:3,:3] = R
    extMat[3,3] = 1
    extMat[:3,3] = tvec
    return R

[[-1.0000000e+00  0.0000000e+00  1.2246468e-16]
 [-0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [-1.2246468e-16 -0.0000000e+00 -1.0000000e+00]]
[[-1.0000000e+00 -0.0000000e+00 -1.2246468e-16]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [ 1.2246468e-16 -0.0000000e+00 -1.0000000e+00]]


In [ ]:
get_extrinsics